In [ ]:
!git clone https://bitbucket.org/jadslim/german-traffic-signs

In [ ]:
!dir german-traffic-signs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
import random
import pickle
import pandas as pd

In [ ]:
np.random.seed(0)  

In [ ]:
with open('german-traffic-signs/train.p', 'rb') as f:
    train_data=pickle.load(f)
with open('german-traffic-signs/valid.p', 'rb') as f:
    val_data=pickle.load(f)
with open('german-traffic-signs/test.p', 'rb') as f:
    test_data=pickle.load(f)

print(type(train_data))

X_train, Y_train = train_data['features'], train_data['labels']

X_val, Y_val = val_data['features'], val_data['labels']

X_test, Y_test = test_data['features'], test_data['labels']


In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
assert(X_train.shape[0]==Y_train.shape[0]), "The number of images is not equal to the number of labels"
assert(X_val.shape[0]==Y_val.shape[0]), "The number of images is not equal to the number of labels"
assert(X_test.shape[0]==Y_test.shape[0]), "The number of images is not equal to the number of labels"
assert(X_train.shape[1:]==(32,32,3)), "The dimensions of the images are not 32,32,3"
assert(X_val.shape[1:]==(32,32,3)), "The dimensions of the images are not 32,32,3"
assert(X_test.shape[1:]==(32,32,3)), "The dimensions of the images are not 32,32,3"

In [ ]:
data = pd.read_csv('german-traffic-signs/signnames.csv')

num_of_samples = []
 
cols = 5
num_classes = 43
 
fig, axs = plt.subplots(nrows=num_classes, ncols = cols, figsize=(5, 50))
fig.tight_layout()
for i in range(cols):
    for j, row in data.iterrows():
        x_selected = X_train[Y_train == j]
        axs[j][i].imshow(x_selected[random.randint(0, len(x_selected - 1)), :, :], cmap=plt.get_cmap("gray"))
        axs[j][i].axis("off")
        if i == 2:
            axs[j][i].set_title(str(j) + "_" + row["SignName"])
            num_of_samples.append(len(x_selected))

#(index, Series)




In [ ]:
print(num_of_samples)
plt.figure(figsize=(12, 4))
plt.bar(range(0, num_classes), num_of_samples)
plt.title("Distribution of the training dataset")
plt.xlabel("Class number")
plt.ylabel("Number of images")

In [ ]:
import cv2

plt.imshow(X_train[1000])
plt.axis("off")
print(X_train[1000].shape)
print(Y_train[1000])

In [ ]:
def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return img

img = grayscale(X_train[1000])
plt.imshow(img)
plt.axis("off")
print(img.shape)

In [ ]:
def equalize(img):
    cv2.equalizeHist(img)
    return img

img = equalize(img)
plt.imshow(img)
plt.axis("off")
print(img.shape)

In [ ]:
def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img/255
    return img

X_train = np.array(list(map(preprocessing, X_train)))
X_val = np.array(list(map(preprocessing, X_val)))
X_test = np.array(list(map(preprocessing, X_test)))

In [ ]:
plt.imshow(X_train[random.randint(0, len(X_train)-1)])
plt.axis("off")
print(X_train.shape)

In [ ]:
X_train = X_train.reshape(34799, 32, 32, 1)
X_test = X_test.reshape(12630, 32, 32, 1)
X_val = X_val.reshape(4410, 32, 32, 1)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

In [ ]:
Y_train = to_categorical(Y_train, 43)
Y_test = to_categorical(Y_test, 43)
Y_val = to_categorical(Y_val, 43)

In [ ]:
def leNet_model():
    model = Sequential()
    model.add(Conv2D(30,(5,5), input_shape=(32,32,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(15, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(Adam(lr=0.01), loss = 'categorical_crossentropy', metrics= ['accuracy'])
    return model

In [ ]:
model = leNet_model()
print(model.summary())

In [ ]:
history = model.fit(X_train, Y_train, epochs = 10, validation_data=(X_val, Y_val), batch_size= 400, verbose = 1, shuffle= 1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training', 'validation'])
plt.title('loss')
plt.xlabel('epoch')